In [63]:
%matplotlib inline
import os
import os.path
import csv
import numpy
import scipy.stats
import scipy.cluster
import itertools
import math
import sys
import sklearn.cluster
import sklearn.decomposition
import sklearn.manifold
import skimage.filters
import pandas
from matplotlib import pylab, mlab
import matplotlib.cm
import matplotlib.patches
import IPython.display
# local modules (in same dir as this notebook)
import plotutils
# import filterutils
import syspathutils


In [64]:
base_path = os.path.realpath(os.path.join(os.path.curdir, '..'))
data_path = os.path.join(base_path, 'samples')
syspathutils.append_to_sys_path('/home/gio/langdev/gitclones/arbimon2-jobs/.env/lib/python2.7/site-packages/')
syspathutils.append_to_sys_path('/home/gio/langdev/gitclones/arbimon2-jobs/lib/')

In [65]:
import a2audio.rec
import a2pyutils.storage
import a2audio.segmentation
import a2pyutils.convert

In [66]:
datasets = {x.replace('sample', '').strip('_'):os.path.join(data_path, x) for x in os.listdir(data_path)}
print '\n'.join("{} : {}".format(x, y) for x, y in datasets.items())

sabana_seca : /home/gio/langdev/gitclones/mine/fltr-audio-seg/samples/sabana_seca
el_verde : /home/gio/langdev/gitclones/mine/fltr-audio-seg/samples/el_verde
validation_data : /home/gio/langdev/gitclones/mine/fltr-audio-seg/samples/validation_data
amarakaeri : /home/gio/langdev/gitclones/mine/fltr-audio-seg/samples/amarakaeri


In [67]:
# dataset = 'smithsonian'
# dataset = 'el_verde1'
# dataset = 'sabana_seca1'
dataset = 'validation_data'
# roi_filepath = ('/home/gio/langdev/gitclones/arbimon2-jobs/synth_output/rois.txt')

roi_filepath = os.path.join(datasets[dataset], 'rois.txt')

rois_data_ori = pandas.read_csv(roi_filepath)
# compute tod and site name from recording filename
rois_data_ori['tod'] = rois_data_ori['rec'].apply(lambda x: int(x.split('.')[0].split('_')[-1].split('-')[0]) if 'undefined' not in x else 0)
# rois_data_ori['site'] = rois_data_ori['rec'].apply(lambda x: int(x.split('/')[0]) if 'undefined' not in x else 0)
rois_data_ori['site'] = rois_data_ori['rec'].apply(lambda x: 1)

print "dataset:{}, sites:{}, recs:{}, rois:{}, rois per rec:{}".format(
    dataset,
    len(rois_data_ori.groupby('site')), 
    len(rois_data_ori.groupby('rec')), 
    len(rois_data_ori), 
    len(rois_data_ori) * 1.0 / len(rois_data_ori.groupby('rec'))
)

dataset:validation_data, sites:1, recs:6, rois:2167, rois per rec:361.166666667


In [68]:
recs = pandas.read_csv(os.path.join(datasets[dataset], 'recs.txt'))
vals = pandas.read_csv(os.path.join(datasets[dataset], 'validations.txt'), index_col=0)
valsbyrec = vals.groupby('rec')

In [74]:
def draw_bounds(X, x0, x1, y0, y1, color):
    for chi, ch in enumerate(color):
        X[y0,x0:(x1+1),chi] = ch
        X[y1,x0:(x1+1),chi] = ch
        X[(y0+1):(y1),x0,chi] = ch
        X[(y0+1):(y1),x1,chi] = ch
    
    return X

dataset_storage = a2pyutils.storage.from_uri(datasets[dataset])
output_storage = dataset_storage

for rec, val_indices in valsbyrec.groups.items():
#     print vals.loc[val_indices].rec
    roi_rec = a2audio.rec.Rec(rec, '', dataset_storage, resample=False)
    spectrum, freqs, times = roi_rec.getSpectrogram()
    specDB = numpy.zeros((spectrum.shape[0], spectrum.shape[1], 3), dtype=numpy.float)
    duration = roi_rec.samples * 1.0 / roi_rec.sample_rate
    max_freq = roi_rec.sample_rate / 2.0
    px2sec = duration / specDB.shape[1]
    px2Hz = max_freq / specDB.shape[0]
    print rec, len(vals), specDB.shape[:2], (21*px2sec, 21*px2Hz), roi_rec.sample_rate
    for val_index in val_indices:
        roi_row = vals.loc[val_index]
        x0, y0 = int(math.floor(roi_row.t0 / px2sec)), int(math.floor(specDB.shape[0] - 1 - roi_row.f1 / px2Hz))
        x1, y1 = int(math.ceil(roi_row.t1 / px2sec)), int(math.ceil(specDB.shape[0] - 1 - roi_row.f0 / px2Hz))
        draw_bounds(specDB, 
                    max(x0,0), 
                    min(x1,specDB.shape[1]-1), 
                    max(y0,0), 
                    min(y1,specDB.shape[0]-1), 
                    (1.0,1.0,1.0)
                   )
    
    with output_storage.open_for_writing("validations-{}.png".format(rec)) as fout:
        pylab.imsave(fout, 
             specDB
        )

EP_16-2013-03-23_03-58.flac 2207 (257, 10327) (0.12192283749948124, 1801.750972762646) 44100
2015-03-23_07-40.p4.flac 2207 (257, 10334) (0.1219276175730598, 1801.750972762646) 44100
1000B-2015-02-10_00-50.flac 2207 (257, 10378) (0.12191655424937368, 1801.750972762646) 44100
Mona_DSG_1014-2013-01-15_23-35.flac 2207 (257, 817) (0.5382580171358629, 408.56031128404663) 10000
Mona_DSG_1014-2013-01-15_23-20.flac 2207 (257, 817) (0.5382580171358629, 408.56031128404663) 10000
Mona_DSG_1014-2013-01-15_23-55.flac 2207 (257, 817) (0.5382580171358629, 408.56031128404663) 10000


In [ ]:
rois_data_ori['f1'] - rois_data_ori['f0']

In [ ]:
spec2samp = roi_rec.original.shape[0] * 1.0 / spectrum.shape[1]
pad=10
samp_x0, samp_x1 = int((x0-pad)*spec2samp), int((x1+pad)*spec2samp+.5)
print f0,f1
roi_clip = roi_rec.original[samp_x0:samp_x1]
pylab.figure(figsize=(15,5))

roi_clip_bpf = filterutils.butter_bandpass_filter(
    roi_rec.original, f0, f1, roi_rec.sample_rate, order=7, passes=4
)[samp_x0:samp_x1]
# roi_clip_bpf =  roi_clip

pylab.plot(roi_clip_bpf)

pylab.figure(figsize=(10,5))
pylab.subplot(1,2,1)
pylab.specgram(roi_clip_bpf)
pylab.subplot(1,2,2)
pylab.specgram(roi_clip)

IPython.display.display(IPython.display.Audio(roi_clip_bpf, rate=roi_rec.sample_rate))

In [ ]:
numpy.array([1j,2,3j,1+4j]).conjugate()

In [ ]:
for i_idx, i in enumerate(idxx):
    print "{}: ".format(i)
    roi_row1 = rois_data_ori.iloc[i]
    (roi1, roi1_bounds), _1, _2 = get_roi(roi_row1, sabana_seca1_sample_storage)
    with region_rois_output.open_for_writing("data-roi-{}.png".format(i)) as fout:
        numpy.save(fout, roi1)


In [ ]:
def get_roi(RR, storage, pad=10):
    roi_rec = a2audio.rec.Rec(RR['rec'], '', storage)
    spectrum, freqs, times = roi_rec.getSpectrogram()
    specDB = a2pyutils.convert.power2decibels(spectrum)
    max_v = specDB.flatten().max()
    duration = roi_rec.samples * 1.0 / roi_rec.sample_rate
    max_freq = roi_rec.sample_rate / 2.0
    px2sec = duration / specDB.shape[1]
    px2Hz = max_freq / specDB.shape[0]

    x0, y1 = RR['x'], specDB.shape[0] - RR['y']
    x1, y0 = x0 + RR['w'], y1 - RR['h']
    
    return crop(specDB, x0, x1, y0, y1, pad), (duration, max_freq, max_v), (px2sec, px2Hz)

def crop(X, x0, x1, y0, y1, pad=10):
    h,w = X.shape
    px0, px1, py0, py1 = max(x0-pad, 0), min(x1+pad, w-1), max(y0-pad, 0), min(y1+pad, h-1)
    roi = X[py0:py1,px0:px1].astype(numpy.float32)
    return roi, (px0, py0, px1, py1)

def correl(X, Y):
    (xh,xw), (yh,yw) = X.shape, Y.shape
    pX = numpy.pad(X, ((0,yh),(0,yw)), mode='constant', constant_values=(float(numpy.median(X)),))
    pY = numpy.pad(Y, ((xh,0),(xw,0)), mode='constant', constant_values=(float(numpy.median(Y)),))
    fX, fY = numpy.fft.fft2(pX), numpy.fft.fft2(pY)
    fCorr = (fX * fY.conjugate())
    fCorr /= abs(fX) * abs(fY)
    Corr = numpy.fft.ifft2(fCorr)
    return abs(Corr)

def roi_dist(roi1, roi2):
    C = correl(roi1, roi2)
#     pylab.imshow(C, interpolation="nearest")
    return C.max()


sabana_seca1_sample_storage = a2pyutils.storage.from_uri('/home/gio/langdev/gitclones/mine/fltr-audio-seg/samples/sabana_seca1_sample/')
region_rois_output = a2pyutils.storage.from_uri('/home/gio/langdev/gitclones/mine/fltr-audio-seg/samples/sabana_seca1_sample/region_bw_700_1900_ymax_2650_3550')
# sabana_seca1_sample_path = '/home/gio/langdev/gitclones/mine/fltr-audio-seg/samples/sabana_seca1_sample/'
# sabana_seca1_sample = os.listdir(sabana_seca1_sample_path)


print "total number of rois : {}".format(len(rois_data_ori))

idxx = range(len(rois_data_ori))
# idxx = [12,13,23, 40]
# idxx = [12, 13, 15, 18, 21, 23, 31, 32, 37, 40, 41, 42, 47, 48]
# idxx = [0, 1, 2, 3, 4, 5]
# pylab.figure(figsize=(21,21))

with region_rois_output.open_for_writing("roi-dist.txt") as fout:
    fout.write("i, j, d\n")
    for i_idx, i in enumerate(idxx):
        print "{}: ".format(i)
        roi_row1 = rois_data_ori.iloc[i]
        (roi1, roi1_bounds), _1, _2 = get_roi(roi_row1, sabana_seca1_sample_storage)
        for j_idx, j in enumerate(idxx[i_idx+1:]):
            roi_row2 = rois_data_ori.iloc[j]
            (roi2, roi2_bounds), _1, _2 = get_roi(roi_row2, sabana_seca1_sample_storage)
            d = roi_dist(roi1, roi2)
            print "   vs {} : sim : {}".format(j, d)
            fout.write("{}, {}, {}\n".format(i, j, d))
# #     with region_rois_output.open_for_writing("roi2-{}.png".format(i)) as fout:
# #         pylab.imsave(fout, 
# #              roi,
# #              origin='lower'
# #         )

In [ ]:
(roi1, roi1_bounds), _1, _2 = get_roi(rois_data_ori.iloc[4], sabana_seca1_sample_storage)
(roi2, roi2_bounds), _1, _2 = get_roi(rois_data_ori.iloc[5], sabana_seca1_sample_storage)
(roi3, roi2_bounds), _1, _2 = get_roi(rois_data_ori.iloc[3], sabana_seca1_sample_storage)
# d = roi_dist(roi1, roi2)
pylab.subplot(1,3,1)
pylab.imshow(roi1)
pylab.subplot(1,3,2)
pylab.imshow(roi2)
pylab.subplot(1,3,3)
pylab.imshow(roi3)
